In [45]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [46]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [47]:
X.shape, y.shape

((1000, 10), (1000,))

In [48]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

(((700, 10), (700,)), ((300, 10), (300,)))

In [49]:
lr = LogisticRegression(C=1, solver='liblinear')
lr.fit(X_train, y_train)
preds = lr.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [50]:
rf = RandomForestClassifier(n_estimators=45, max_depth=3)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.63      0.76        30

    accuracy                           0.96       300
   macro avg       0.96      0.81      0.87       300
weighted avg       0.96      0.96      0.96       300



In [51]:
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [52]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=34)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(X_train_res, return_counts=True)

(array([-5.51207083, -5.14120195, -4.69421946, ...,  5.71550663,
         5.79413843,  6.02415826]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [53]:
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train_res, y_train_res)
preds = xgb.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       270
           1       0.76      0.87      0.81        30

    accuracy                           0.96       300
   macro avg       0.87      0.92      0.90       300
weighted avg       0.96      0.96      0.96       300



## Experiments tracking with MLflow

In [54]:
models = [
    (
        'Logistic Regression',
        LogisticRegression(C=1, solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ), (
        'Random Forest',
        RandomForestClassifier(n_estimators=45, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ), (
        'XGB 1',
        XGBClassifier(eval_metric='logloss'),
        (X_train, y_train),
        (X_test, y_test)
    ), (
        'XGB 2',
        XGBClassifier(eval_metric='logloss'),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [55]:
metrics = {}

for i, mod in enumerate(models):
    model_name = mod[0]
    model = mod[1]
    model.fit(mod[2][0], mod[2][1])
    preds = model.predict(mod[3][0])
    report = classification_report(mod[3][1], preds, output_dict=True)
    metrics[i] = report

metrics

{0: {'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669724,
   'support': 270},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.9132610508757297,
   'support': 300}},
 1: {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300},
  'weighted avg': {'precision'

In [61]:
import mlflow
mlflow.set_experiment("multiple_models")
mlflow.set_registry_uri(uri="http://127.0.0.1:5000/")

for i, model in enumerate(models):
    model_name = model[0]
    model = model[1]
    report = metrics[i]
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param('model', model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_0', report['0']['recall'])
        mlflow.log_metric('recall_1', report['1']['recall'])
        mlflow.log_metric('f1-score', report['macro avg']['f1-score'])
        if 'xgb' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')

2024/08/18 07:29:41 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/18 07:29:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000//#/experiments/338671186205244746/runs/8c8035d4b4e74bf7988d9219fb762acb.
2024/08/18 07:29:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000//#/experiments/338671186205244746.
2024/08/18 07:29:44 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/18 07:29:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000//#/experiments/338671186205244746/runs/b06ee82cc38b4d8490024bc4752f24bb.
2024/08/18 07:29:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000//#/experiments/